In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0,'../../')
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 2000)
sys.path.append('../../ArgCausalDisco/utils/')
from ArgCausalDisco.utils.plotting import *
print(sys.path)

save_figs = True
debug = False
datasets = ['cancer', 'earthquake', 'survey', 'asia']
dags_nodes_map = {'asia':8, 'cancer':5, 'earthquake':5, 'sachs':11, 'survey':6, 'alarm':37, 'child':20, 'insurance':27, 'hailfinder':56, 'hepar2':70}
dags_arcs_map = {'asia':8, 'cancer':4, 'earthquake':4, 'sachs':17, 'survey':6, 'alarm':46, 'child':25, 'insurance':52, 'hailfinder':66, 'hepar2':123}
methods = ['Random', 'FGS', 'NOTEARS-MLP', 'MPC', 'ABAPC (Ours)', 'ABAPC (ASPforABA)']
names_dict = {'fgs':'FGS', 'nt':'NOTEARS-MLP', 'mpc':'MPC', 'random':'Random', 'abapc':'ABAPC (Ours)', 'pure_abapc': 'ABAPC (ASPforABA)'}
symbols_dict = {'abapc':'triangle-down-dot','fgs':'triangle-up-dot','nt':'pentagon-dot','mpc':'hexagon2-dot', 'random':'x', 'pure_abapc': 'star-dot'}  
colors_dict = {'abapc':sec_blue,'fgs':sec_orange,'nt':main_purple,'mpc':main_green,'random':'grey', 'pure_abapc':'black'}
version = 'bnlearn_50rep' ## for 5000 samples
# version = 'bnlearn_dag_v5_2000' ## for 2000 samples

In [ ]:
version_cpdag = version+'_cpdag'
all_sum = pd.read_csv(f"../../results_pure_aba/stored_results_{version_cpdag}.csv")
columns=['dataset', 'model', 'elapsed_mean', 'elapsed_std', 'nnz_mean', 'nnz_std', 
        'fdr_mean', 'fdr_std', 'tpr_mean', 'tpr_std', 'fpr_mean', 'fpr_std', 
        'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
        'F1_mean', 'F1_std', 'shd_mean', 'shd_std', 
        'SID_low_mean', 'SID_low_std', 'SID_high_mean', 'SID_high_std'
        ]
assert len(columns) == len(all_sum.columns)
all_sum.columns = columns

all_sum['n_edges'] = all_sum['dataset'].map(dags_arcs_map)
all_sum['n_nodes'] = all_sum['dataset'].map(dags_nodes_map)
for var in ['shd','SID_low','SID_high']:
    all_sum['p_'+var+'_mean'] = all_sum[var+'_mean'].astype(float)/all_sum['n_edges'].astype(int)
    all_sum['p_'+var+'_std'] = all_sum[var+'_std'].astype(float)/all_sum['n_edges'].astype(int)
all_sum['dataset'] = [a.upper() for a in all_sum["dataset"].astype(str)]
all_sum['dataset'] = all_sum['dataset'] +np.repeat("<br> |V|=",len(all_sum))+ all_sum["n_nodes"].astype(str)+np.repeat(", |E|=",len(all_sum))+\
                     all_sum["n_edges"].astype(str)

double_bar_chart_plotly(all_sum, ['p_SID_low','p_SID_high'], names_dict, colors_dict, methods, save_figs=save_figs, output_name="./Fig.2_SID_cpdag.html", debug=False, range_y1=[0,6], range_y2=[0,6])#

In [ ]:
plot_runtime(all_sum, ['n_nodes'], "n_nodes <= 10", 
                     names_dict, symbols_dict, colors_dict, ['mpc', 'abapc', 'pure_abapc'],
                         share_y=False, save_figs=save_figs, 
                            output_name="./Fig.3_runtime.html", debug=False, font_size=20)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Enable Kaleido export
pio.kaleido.scope.default_format = "png"

# Load your dataset
df = all_sum

# Optional: Clean HTML line breaks in 'dataset'
df['dataset'] = df['dataset'].str.replace('<br>', '\n', regex=False)

# ✅ Filter only ABAPC models
models_of_interest = ['ABAPC (ASPforABA)', 'ABAPC (Ours)']
df_filtered = df[df['model'].isin(models_of_interest)]

# Create grouped bar plot
fig = px.bar(
    df_filtered,
    x='dataset',
    y='elapsed_mean',
    color='model',
    barmode='group',
    error_y='elapsed_std',
    title="Elapsed Time (Mean ± Std) for ABAPC Models by Dataset",
    labels={'elapsed_mean': 'Elapsed Time (Mean)', 'dataset': 'Dataset'},
    height=600
)

fig.update_layout(
    xaxis_tickangle=-0,
    bargap=0.25,
    legend_title_text='Model',
    template='plotly_white'
)

# Show the plot
fig.show()

# Export to image
fig.write_image("elapsed_time_grouped_by_dataset.png", scale=2)
